# Word Embeddings

Word Embeddings are dense vectors representations of words. Compare this to sparse vectors like One-Hot Encoding.

In [1]:
sentence = "the quick brown fox jumped over the lazy dog"
words = sentence.split(' ')

Let's look at the individual words in our vocabulary:

In [2]:
vocabulary = list(set(words))
print(vocabulary)

['quick', 'dog', 'the', 'lazy', 'over', 'jumped', 'brown', 'fox']


In [3]:
# Number of words in our vocabulary
len(vocabulary)

8

# One Hot Encoding

The vocabulary size is seen above. Now we can one-hot encode the vocabulary words. Let's try using Scikit-Learn One-Hot encoding:

In [4]:
from sklearn.preprocessing import LabelBinarizer
enc = LabelBinarizer()
enc.fit(vocabulary)
enc.classes_

array(['brown', 'dog', 'fox', 'jumped', 'lazy', 'over', 'quick', 'the'],
      dtype='<U6')

Here's what our one-hot encoded vocabulary looks like.

In [5]:
# Might want to have another diagram here which has the words as column headers
print(enc.classes_)
print(enc.transform(vocabulary))

['brown' 'dog' 'fox' 'jumped' 'lazy' 'over' 'quick' 'the']
[[0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]]


The issue with sparse one-hot encoding is that the vectors are very large 
and we have a very sparse representation of the vectors. As you can see there
are a lot of zeroes. For example, the popular data set WikiText-103 has 267,000
words in the vocabulary. This means around 267,000 zeroes in each vector with
one-hot encoding.

We should try to find a smaller encoding for our dataset. Let's try a denser vector using a
Word Embedding.

In [6]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

# Show what a trigram looks like

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


# N-Gram Language Model

An N-Gram is a sequence of words as in a sentence. This is useful because it gives us some context to train a deep learning classifier. 

For a detailed post visit: https://www.microsoft.com/developerblog/2015/11/29/feature-representation-for-text-analyses/

Here's what a diagram of our n-gram deep learning model would look like:
<img src="word_embedding_model.png">

Training based on surrounding words. We want to predict the probability of a word based on the words around it.

In [7]:
# Add imports here
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [8]:
learning_rate = 0.001
losses = []
loss_function = nn.NLLLoss()  # negative log likelihood
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
import sys
! {sys.prefix}/bin/pip install tqdm --yes


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes


In [10]:
from tqdm import tqdm

for epoch in range(10):
    total_loss = 0

    for context, target in tqdm(trigrams):
        # (['When', 'forty'], 'winters')
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    # add progress bar with epochs
print(losses)  # The loss decreased every iteration over the training data!

100%|██████████| 113/113 [00:00<00:00, 272.94it/s]

[518.3796315193176, 515.9071173667908, 513.4507246017456, 511.00923919677734, 508.5816111564636, 506.1678500175476, 503.76671600341797, 501.37714862823486, 498.9977984428406, 496.62781953811646]


In [11]:
model.eval()

NGramLanguageModeler(
  (embeddings): Embedding(97, 10)
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)

In [20]:
with torch.no_grad():
    model([[('When', 'forty'), ]])

AttributeError: 'list' object has no attribute 'contiguous'

In [12]:
# We should show the embedding vector for a words

# We should show the prediction given two words (probabilities and the highest prediction -- look up torch argmax )

# Next Steps
* RNN/LSTM/BiLSTM
* Why memory? It gives more context for a sentence. Less rigid.
* Pointer to GloVe word embedding
* https://github.com/fastai/word-embeddings-workshop/blob/master/Word%20Embeddings.ipynb


# Exercise: Continuous Bag of Words
Continuous Bag of Words is a model that tries to predict a word based on a few word before and after the word.

* Not sequential
* Not necessarily probabilistic
* Used for Pretraining a word embedding

TODO: Add Image Here of words before and after

In [13]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [14]:
# create your model and train.  here are some functions to help you make
# the data ready for use by your module

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)  # example

tensor([ 18,   1,  42,  15])

In [15]:
class CBOW(nn.Module):
    def __init__(self):
        pass

    def forward(self, inputs):
        pass
# Solutions in another directory?
# 


References: 
* https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
* https://github.com/fastai/word-embeddings-workshop/blob/master/Word%20Embeddings.ipynb